![Hand with calculator](calculator.png "Calculator")

Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


# TASK 1: TOP INDUSTRIES
Your task is to first identify the **three best-performing industries based** on the **number of new unicorns created over the last three years** (2019, 2020, and 2021) combined. >> THIS RESULT NEEDS TO BE SAVES AS A Common Table Expression CTE

In [4]:
-- COMMON TABLE EXPRESSION TOP INDUSTRIES START --
WITH top_industries AS ( 
SELECT subq.*
FROM ( ---- SUBQUERY BEGINNING ----
    SELECT 
        ind.industry
        ,EXTRACT(year FROM da.date_joined) AS year
        ,COUNT(da.company_id) AS numb_unicorns
        ,RANK() OVER(PARTITION BY EXTRACT(year FROM da.date_joined) 
                     ORDER BY COUNT(da.company_id) DESC) AS rank
        
    FROM dates AS da
        INNER JOIN industries AS ind
        USING (company_id)
    WHERE 
        EXTRACT(year FROM da.date_joined) IN (2019,2020,2021)
    GROUP BY ind.industry, year
    ORDER BY
        year DESC
        ,numb_unicorns DESC) AS subq
         ---- SUBQUERY END ----
WHERE subq.rank IN (1,2,3)
-- COMMON TABLE EXPRESSION TOP INDUSTRIES END --)
)

SELECT *
FROM top_industries;

,industry,year,numb_unicorns,rank
0,Fintech,2021,138,1
1,Internet software & services,2021,119,2
2,E-commerce & direct-to-consumer,2021,47,3
3,Internet software & services,2020,20,1
4,E-commerce & direct-to-consumer,2020,16,2
5,Fintech,2020,15,3
6,Fintech,2019,20,1
7,Artificial intelligence,2019,14,2
8,Internet software & services,2019,13,3


# TASK 2:
From there, you will write a query to return the **industry**, the **year**, the **number of companies** in these industries that became unicorns each year in 2019, 2020, and 2021, along with the **average valuation per industry per year**, converted to billions of dollars and rounded to two decimal places!

In [7]:
WITH yearly_rankings AS (
SELECT
    EXTRACT(year FROM da.date_joined) AS year
    ,ind.industry AS industry
    ,COUNT(da.company_id) AS num_unicorns
    ,AVG(fu.valuation) AS average_valuation


    
FROM dates AS da
INNER JOIN funding AS fu
    USING (company_id)
INNER JOIN industries AS ind
    USING (company_id)
    
GROUP BY
    EXTRACT(year FROM da.date_joined)
    ,ind.industry
    
)
 
SELECT *
FROM yearly_rankings;


,year,industry,num_unicorns,average_valuation
0,2017,Other,4,1.750000e+09
1,2011,Internet software & services,1,6.000000e+09
2,2016,E-commerce & direct-to-consumer,6,6.666667e+09
3,2014,Hardware,2,3.500000e+09
4,2019,E-commerce & direct-to-consumer,12,2.583333e+09
...,...,...,...,...
119,2021,Cybersecurity,27,2.518519e+09
120,2022,Health,7,1.857143e+09
121,2015,Fintech,2,5.500000e+09
122,2014,Health,1,4.000000e+09


# TASK 3:
In the final query you should select the industry, year, num_unicorns, and the average of average_valuation divided by one billion, rounding to two decimal places and aliasing as average_valuation_billions;

In [6]:
-- COMMON TABLE EXPRESSION TOP INDUSTRIES START --
WITH top_industries AS ( 
SELECT subq.*
FROM ( ---- SUBQUERY BEGINNING ----
    SELECT 
        ind.industry
        ,EXTRACT(year FROM da.date_joined) AS year
        ,COUNT(da.company_id) AS numb_unicorns
        ,RANK() OVER(PARTITION BY EXTRACT(year FROM da.date_joined) 
                     ORDER BY COUNT(da.company_id) DESC) AS rank
        
    FROM dates AS da
        INNER JOIN industries AS ind
        USING (company_id)
    WHERE 
        EXTRACT(year FROM da.date_joined) IN (2019,2020,2021)
    GROUP BY ind.industry, year
    ORDER BY
        year DESC
        ,numb_unicorns DESC) AS subq
         ---- SUBQUERY END ----
WHERE subq.rank IN (1,2,3)
-- COMMON TABLE EXPRESSION TOP INDUSTRIES END --)
)



SELECT
    EXTRACT(year FROM da.date_joined) AS year
    ,ind.industry AS industry
    ,COUNT(da.company_id) AS num_unicorns
    ,ROUND(AVG(fu.valuation)/100,2) AS average_valuation_billions

    
FROM dates AS da
INNER JOIN funding AS fu
    USING (company_id)
INNER JOIN industries AS ind
    USING (company_id)

WHERE
    EXTRACT(year FROM da.date_joined) IN (2019,2020,2021)
    AND industry IN (SELECT industry FROM top_industries)
    
GROUP BY
    EXTRACT(year FROM da.date_joined)
    ,ind.industry

ORDER BY industry DESC, year DESC
    
;

,year,industry,num_unicorns,average_valuation_billions
0,2021,Internet software & services,119,21512605.04
1,2020,Internet software & services,20,43500000.00
2,2019,Internet software & services,13,42307692.31
3,2021,Fintech,138,27536231.88
4,2020,Fintech,15,43333333.33
5,2019,Fintech,20,68000000.00
6,2021,E-commerce & direct-to-consumer,47,24680851.06
7,2020,E-commerce & direct-to-consumer,16,40000000.00
8,2019,E-commerce & direct-to-consumer,12,25833333.33
9,2021,Artificial intelligence,36,14166666.67
